# YouTube Playlist Video Downloader & Transcript Fetcher

Dieses Notebook verwendet die Bibliothek `pytubefix`, um:
1.  Eine YouTube-Playlist-URL zu analysieren.
2.  Die ersten N Videos aus dieser Playlist herunterzuladen (höchste verfügbare progressive Auflösung).
3.  Zu versuchen, die deutschen Transkripte (Untertitel) für diese Videos abzurufen und als `.srt`-Dateien zu speichern.

**Bibliothek:** `pytubefix` (ein Fork von `pytube`, der oft besser mit aktuellen YouTube-Änderungen umgehen kann).

**Hinweis:** Die Verfügbarkeit von Transkripten hängt davon ab, ob sie für das jeweilige Video auf YouTube existieren und ob `pytubefix` sie korrekt abrufen kann.

## Teil 0: Setup

Installation von `pytubefix` und Import der benötigten Module.

In [ ]:
# Installiere pytubefix
!pip install -q pytubefix

In [ ]:
from pytubefix import Playlist, YouTube
from pytubefix.exceptions import PytubeFixError, VideoUnavailable # Import spezifischer Exceptions
import os # Für Dateipfade und Ordnererstellung
import re # Für das Bereinigen von Dateinamen

## Teil 1: Konfiguration

Definieren Sie die Playlist-URL, die Anzahl der zu verarbeitenden Videos und das Zielverzeichnis.

In [ ]:
# --- Konfiguration --- 
playlist_url = "https://www.youtube.com/watch?v=Rd8SmbOG50Q&list=PL4A2F331EE86DCC22" # Tagesschau Playlist
num_videos_to_process = 5 # Anzahl der Videos, die wir von Anfang an nehmen
download_folder = "tagesschau_downloads" # Ordner für Videos und Transkripte
# ---- Ende Konfiguration ---

In [ ]:
# Erstelle den Download-Ordner, falls er nicht existiert
if not os.path.exists(download_folder):
    os.makedirs(download_folder)
    print(f"Ordner '{download_folder}' erstellt.")
else:
    print(f"Ordner '{download_folder}' existiert bereits.")

## Teil 2: Playlist-Informationen abrufen

Wir erstellen ein Playlist-Objekt und holen uns die URLs der ersten N Videos.

In [ ]:
try:
    playlist = Playlist(playlist_url)
    print(f"Playlist Titel: {playlist.title}")
    print(f"Gesamtzahl Videos in Playlist (laut Metadaten): {playlist.length}")

    # Hole die URLs der Videos (kann bei sehr langen Playlists dauern)
    # Wir nehmen nur die ersten 'num_videos_to_process' URLs
    video_urls = playlist.video_urls[:num_videos_to_process]

    print(f"Die ersten {len(video_urls)} Video-URLs werden verarbeitet:")
    for i, url in enumerate(video_urls):
        print(f"  {i+1}: {url}")

except Exception as e:
    print(f"Fehler beim Abrufen der Playlist-Informationen: {e}")
    video_urls = [] # Leere Liste, damit der nächste Schritt nicht fehlschlägt

## Teil 3: Videos herunterladen und Transkripte abrufen

Wir iterieren durch die ausgewählten Video-URLs, laden das Video herunter und versuchen, das deutsche Transkript zu speichern.

In [ ]:
def sanitize_filename(filename):
    """Entfernt ungültige Zeichen aus einem Dateinamen."""
    # Entferne Zeichen, die in Windows/Linux/Mac problematisch sind
    sanitized = re.sub(r'[\/*?"<>|:]', "", filename)
    # Ersetze Leerzeichen (optional, verbessert Lesbarkeit)
    # sanitized = sanitized.replace(" ", "_")
    return sanitized

if not video_urls:
    print("Keine Video-URLs zum Verarbeiten vorhanden.")
else:
    print(f"\n--- Starte Download & Transkript-Abruf für {len(video_urls)} Videos ---")

    for i, url in enumerate(video_urls):
        print(f"\nVerarbeite Video {i+1}/{len(video_urls)}: {url}")
        try:
            yt = YouTube(url)
            video_title = yt.title
            safe_video_title = sanitize_filename(video_title) # Bereinigter Titel für Dateinamen
            print(f"  Titel: {video_title}")

            # --- Video Download ---
            try:
                # Wähle den Stream mit der höchsten Auflösung (progressiv = Video+Audio)
                stream = yt.streams.get_highest_resolution()
                if stream:
                    print(f"  Lade Video herunter (Auflösung: {stream.resolution}, Typ: {stream.mime_type})...")
                    # Definiere den vollständigen Dateipfad
                    video_filename = f"{safe_video_title}.{stream.subtype}" # z.B. .mp4
                    video_filepath = os.path.join(download_folder, video_filename)

                    if os.path.exists(video_filepath):
                        print(f"  Video '{video_filename}' existiert bereits. Überspringe Download.")
                    else:
                        stream.download(output_path=download_folder, filename=video_filename)
                        print(f"  Video erfolgreich heruntergeladen: {video_filename}")
                else:
                    print("  Kein geeigneter Video-Stream zum Download gefunden.")
            except Exception as download_error:
                print(f"  Fehler beim Video-Download: {download_error}")

            # --- Transkript Abruf --- 
            try:
                captions = yt.captions
                if captions:
                    print(f"  Verfügbare Untertitel-Sprachen: {list(captions.keys())}")
                    # Versuche, das deutsche Transkript zu bekommen ('de' ist der Sprachcode)
                    german_caption = captions.get('de')

                    if german_caption:
                        print("  Deutsches Transkript gefunden. Rufe SRT ab...")
                        srt_captions = german_caption.generate_srt_captions()

                        # Speichere das Transkript als .srt Datei
                        transcript_filename = f"{safe_video_title}_de.srt"
                        transcript_filepath = os.path.join(download_folder, transcript_filename)

                        if os.path.exists(transcript_filepath):
                            print(f"  Transkript '{transcript_filename}' existiert bereits. Überspringe Speichern.")
                        else:
                            with open(transcript_filepath, 'w', encoding='utf-8') as f:
                                f.write(srt_captions)
                            print(f"  Deutsches Transkript gespeichert: {transcript_filename}")
                    else:
                        print("  Kein deutsches Transkript verfügbar für dieses Video.")
                else:
                    print("  Keine Untertitel/Transkripte für dieses Video verfügbar.")

            except Exception as transcript_error:
                 print(f"  Fehler beim Abrufen/Verarbeiten des Transkripts: {transcript_error}")

        # --- Fehlerbehandlung für das YouTube Objekt selbst ---
        except VideoUnavailable:
             print(f"  FEHLER: Video {url} ist nicht verfügbar (privat, gelöscht o.ä.).")
        except PytubeFixError as e:
             print(f"  FEHLER bei der Verarbeitung von {url}: {e}")
        except Exception as e:
             print(f"  Ein unerwarteter FEHLER ist bei {url} aufgetreten: {e}")

print("\n--- Verarbeitung abgeschlossen ---")

## Teil 4: Fazit

Die Videos (falls erfolgreich) und die deutschen Transkripte (falls verfügbar und erfolgreich) sollten nun im Ordner `tagesschau_downloads` im Dateisystem dieses Colab-Notebooks zu finden sein.

**Mögliche nächste Schritte:**
*   Die heruntergeladenen `.srt`-Dateien können weiterverarbeitet werden (z.B. Text extrahieren, analysieren).
*   Die Anzahl der heruntergeladenen Videos (`num_videos_to_process`) kann angepasst werden.
*   Die Fehlerbehandlung kann verfeinert werden.
*   Andere Stream-Qualitäten oder Formate könnten ausgewählt werden (z.B. nur Audio).